In [1]:
import keras
from keras.models import load_model
#from keras.datasets import cifar10
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, BatchNormalization, Activation, Flatten, Dropout,GlobalAveragePooling2D
from keras.models import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import Adam
from keras.utils import np_utils
from keras.utils.np_utils import to_categorical
import os
import pickle
import numpy as np
import h5py
from keras import backend as K
from keras.optimizers import SGD
from keras.callbacks import LearningRateScheduler
import pandas as pd
from PIL import Image

Using TensorFlow backend.


In [3]:
train=np.load("/mnt/Doc/256/picture.npy")
#train_label=np.load("/mnt/Doc/reserch1/train_type.npy")
##test=np.load("/mnt/Doc/512pix_to_128pix/npy/128pix_testpicture.npy")
val = np.load("/mnt/Doc/256/picture_val.npy")
#test_label=np.load("/mnt/Doc/reserch1/test_type.npy")

#x_train = train.astype('float32')
#x_test = test.astype('float32')
#x_val = val.astype('float32')
train /= 255
#x_test /= 255
val /=255
#del train
#del test
#del val

In [15]:
input_img = Input(shape=(256, 256, 3))

x = Conv2D(4, (7, 7), padding='same')(input_img)
x = Activation('relu')(x)
#x = BatchNormalization()(x)
#x = Dropout(0.25)(x)

x = MaxPooling2D((2, 2), padding='same')(x)

x = Conv2D(8, (5, 5), padding='same')(x)
x = Activation('relu')(x)
#x = BatchNormalization()(x)
#x = Dropout(0.25)(x)

x = MaxPooling2D((2, 2), padding='same')(x)

x = Conv2D(16, (3, 3), padding='same')(x)
x = Activation('relu')(x)
#x = BatchNormalization()(x)
#x = Dropout(0.25)(x)

x = MaxPooling2D((2, 2), padding='same')(x)

#x = Conv2D(32, (3, 3), padding='same')(x)
#x = Activation('relu')(x)
#x = BatchNormalization()(x)
#x = Dropout(0.25)(x)

#encoded = MaxPooling2D((2, 2), padding='same')(x)

#x = Conv2D(32, (3, 3), padding='same')(encoded)
#x = Activation('relu')(x)
#x = BatchNormalization()(x)
#x = Dropout(0.25)(x)

#x = UpSampling2D((2, 2))(x)

x = Conv2D(16, (3, 3), padding='same')(x)
x = Activation('relu')(x)
#x = BatchNormalization()(x)
x = Dropout(0.25)(x)

x = UpSampling2D((2, 2))(x)

x = Conv2D(8, (5, 5), padding='same')(x)
x = Activation('relu')(x)
#x = BatchNormalization()(x)
x = Dropout(0.25)(x)

x = UpSampling2D((2, 2))(x)

x = Conv2D(4, (7, 7), padding='same')(x)
x = Activation('relu')(x)
#x = BatchNormalization()(x)
x = Dropout(0.25)(x)

x = UpSampling2D((2, 2))(x)

x = Conv2D(3, (3, 3), padding='same')(x)

decoded = Activation('relu')(x)
model = Model(input_img, decoded)
model.compile(optimizer='adam', loss='mean_squared_error')
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 256, 256, 3)       0         
_________________________________________________________________
conv2d_31 (Conv2D)           (None, 256, 256, 4)       592       
_________________________________________________________________
activation_31 (Activation)   (None, 256, 256, 4)       0         
_________________________________________________________________
dropout_27 (Dropout)         (None, 256, 256, 4)       0         
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 128, 128, 4)       0         
_________________________________________________________________
conv2d_32 (Conv2D)           (None, 128, 128, 8)       808       
_________________________________________________________________
activation_32 (Activation)   (None, 128, 128, 8)       0         
__________

In [16]:
modelCheckpoint = ModelCheckpoint(filepath ="/mnt/Doc/weight/Autoencoder/256/2019_05_08__04c.h5",
                                  monitor = "val_loss",
                                  verbose=1,
                                  save_best_only = True,
                                  save_weights_only = False,
                                  mode = "min",
                                  period =1)

In [17]:
history = model.fit(train, train,
                    batch_size=16,
                    epochs=100,
                    verbose=1,
                    validation_data=(val, val),
                    callbacks=[modelCheckpoint],
                    shuffle=True)

Train on 6000 samples, validate on 500 samples
Epoch 1/100
6000/6000 [==============================] - 24s 4ms/step - loss: 0.0745 - val_loss: 0.0257

Epoch 00001: val_loss improved from inf to 0.02569, saving model to /mnt/Doc/weight/Autoencoder/256/2019_05_08__04c.h5
Epoch 2/100
6000/6000 [==============================] - 24s 4ms/step - loss: 0.0218 - val_loss: 0.0248

Epoch 00002: val_loss improved from 0.02569 to 0.02476, saving model to /mnt/Doc/weight/Autoencoder/256/2019_05_08__04c.h5
Epoch 3/100
6000/6000 [==============================] - 24s 4ms/step - loss: 0.0164 - val_loss: 0.0234

Epoch 00003: val_loss improved from 0.02476 to 0.02344, saving model to /mnt/Doc/weight/Autoencoder/256/2019_05_08__04c.h5
Epoch 4/100
6000/6000 [==============================] - 23s 4ms/step - loss: 0.0145 - val_loss: 0.0237

Epoch 00004: val_loss did not improve from 0.02344
Epoch 5/100
6000/6000 [==============================] - 23s 4ms/step - loss: 0.0140 - val_loss: 0.0229

Epoch 00005:

KeyboardInterrupt: 

In [4]:
class ImageDataGenerator(object):
    def __init__(self):
        self.reset()

    def reset(self):
        self.images = []
        self.labels = []
    #file_pathはメタCSVでーたのこと
    def flow_from_directory(self, file_path, classes, batch_size=1):
        blunc=np.empty((batch_size, 2048, 2048, 3), np.float32)
        blunc_l=np.empty((batch_size), np.float32)
        # LabelEncode(classをint型に変換)するためのdict\n",
        classes = {v: i for i, v in enumerate(sorted(classes))}
        file_path = pd.read_csv(file_path)
        boss_dir = "/mnt/Doc"
        while True:
            # ディレクトリから画像のパスを取り出す
            count = 0
            for i in range(len(file_path)):
                # 画像を読み込みRGBへの変換、Numpyへの変換を行い、配列(self.iamges)に格納
                path = file_path.loc[i]["path1"]
                #print(i)

                with Image.open(path) as f:
                    self.images.append((np.asarray(f.convert('RGB'), dtype=np.float32))/255)
                
                # ファイル名からラベルを取り出し、配列(self.labels)に格納
                #_, y = path.stem.split('_')
                label = file_path.loc[i]["type"]
                self.labels.append(to_categorical(classes[label],len(classes)))
                # ここまでを繰り返し行い、batch_sizeの数だけ配列(self.iamges, self.labels)に格納
                # batch_sizeの数だけ格納されたら、戻り値として返し、配列(self.iamges, self.labels)を空にする
                if len(self.images)== batch_size:
                    #print(len(self.images))
                    inputs = np.asarray(self.images,dtype=np.float32) 
                    targets = np.asarray(self.images,dtype=np.float32)
                    #print(len(self.images))
                    count+=1
                    #if count%1000==0:
                        #print("n*1000!")
                    self.reset()
                    yield targets, inputs
   

In [5]:
x = pd.read_csv("/mnt/Doc/2048pix_tissue_all/RGBmean<190.csv")
train_dir = "/mnt/Doc/2048pix_tissue_all/val_400.csv"
test_dir = "/mnt/Doc/2048pix_tissue_all/val_400.csv"
train_len = len(pd.read_csv(train_dir))
print(train_len)
test_len = len(pd.read_csv(test_dir))
test_datagen = ImageDataGenerator()
train_datagen = ImageDataGenerator()
classes =["PI","PP","TRU"]
batchsize = 1
model.fit_generator(
    generator=train_datagen.flow_from_directory(train_dir,classes),
    steps_per_epoch=400/1,
    epochs=1000,
    verbose=2,
    validation_data=test_datagen.flow_from_directory(test_dir,classes),
    validation_steps=400/1,
    callbacks = [modelCheckpoint]
    )

400
Epoch 1/1000
 - 643s - loss: 0.0829 - val_loss: 0.0721

Epoch 00001: val_loss improved from inf to 0.07213, saving model to /mnt/Doc/weight/Autoencoder/2048/2019_05_07.h5
Epoch 2/1000
 - 632s - loss: 0.0305 - val_loss: 0.0213

Epoch 00002: val_loss improved from 0.07213 to 0.02133, saving model to /mnt/Doc/weight/Autoencoder/2048/2019_05_07.h5
Epoch 3/1000
 - 632s - loss: 0.0230 - val_loss: 0.0224

Epoch 00003: val_loss did not improve from 0.02133
Epoch 4/1000
 - 632s - loss: 0.0247 - val_loss: 0.0164

Epoch 00004: val_loss improved from 0.02133 to 0.01638, saving model to /mnt/Doc/weight/Autoencoder/2048/2019_05_07.h5
Epoch 5/1000


KeyboardInterrupt: 